In [17]:
from pathlib import Path
import csv
import pandas as pd
import requests
import json

In [18]:
file_path = Path("worldcities.csv")
city_df = pd.read_csv(file_path)

city_df

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6850,139.7514,Japan,JP,JPN,Tōkyō,primary,35676000.0,1392685764
1,New York,New York,40.6943,-73.9249,United States,US,USA,New York,NaN,19354922.0,1840034016
2,Mexico City,Mexico City,19.4424,-99.1310,Mexico,MX,MEX,Ciudad de México,primary,19028000.0,1484247881
3,Mumbai,Mumbai,19.0170,72.8570,India,IN,IND,Mahārāshtra,admin,18978000.0,1356226629
4,São Paulo,Sao Paulo,-23.5587,-46.6250,Brazil,BR,BRA,São Paulo,admin,18845000.0,1076532519
...,...,...,...,...,...,...,...,...,...,...,...
15488,Timmiarmiut,Timmiarmiut,62.5333,-42.2167,Greenland,GL,GRL,Kujalleq,NaN,10.0,1304206491
15489,Cheremoshna,Cheremoshna,51.3894,30.0989,Ukraine,UA,UKR,Kyyivs’ka Oblast’,NaN,0.0,1804043438
15490,Ambarchik,Ambarchik,69.6510,162.3336,Russia,RU,RUS,Sakha (Yakutiya),NaN,0.0,1643739159
15491,Nordvik,Nordvik,74.0165,111.5100,Russia,RU,RUS,Krasnoyarskiy Kray,NaN,0.0,1643587468


In [51]:
city_df["CO"] = " "
city_df["NO2"] = " "
city_df["O3"] = " "
city_df["PM10"] = " "
city_df["PM25"] = " "
city_df["SO2"] = " "
city_df["AQI"] = " "

city_df

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id,CO,NO2,OZONE,PM10,PM25,SO2,AQI,O3
0,Tokyo,Tokyo,35.6850,139.7514,Japan,JP,JPN,Tōkyō,primary,35676000.0,1392685764,,,,,,,,
1,New York,New York,40.6943,-73.9249,United States,US,USA,New York,NaN,19354922.0,1840034016,,,,,,,,
2,Mexico City,Mexico City,19.4424,-99.1310,Mexico,MX,MEX,Ciudad de México,primary,19028000.0,1484247881,,,,,,,,
3,Mumbai,Mumbai,19.0170,72.8570,India,IN,IND,Mahārāshtra,admin,18978000.0,1356226629,,,,,,,,
4,São Paulo,Sao Paulo,-23.5587,-46.6250,Brazil,BR,BRA,São Paulo,admin,18845000.0,1076532519,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15488,Timmiarmiut,Timmiarmiut,62.5333,-42.2167,Greenland,GL,GRL,Kujalleq,NaN,10.0,1304206491,,,,,,,,
15489,Cheremoshna,Cheremoshna,51.3894,30.0989,Ukraine,UA,UKR,Kyyivs’ka Oblast’,NaN,0.0,1804043438,,,,,,,,
15490,Ambarchik,Ambarchik,69.6510,162.3336,Russia,RU,RUS,Sakha (Yakutiya),NaN,0.0,1643739159,,,,,,,,
15491,Nordvik,Nordvik,74.0165,111.5100,Russia,RU,RUS,Krasnoyarskiy Kray,NaN,0.0,1643587468,,,,,,,,


In [48]:
usa_df = city_df.loc[city_df["country"] == "United States"]

usa_df = usa_df.rename(columns={"admin_name": "state"})
usa_df["population"] = usa_df["population"].astype("int64")

usa_df

,city,city_ascii,lat,lng,country,iso2,iso3,state,capital,population,id,CO,NO2,OZONE,PM10,PM25,SO2,AQI
1,New York,New York,40.6943,-73.9249,United States,US,USA,New York,NaN,19354922,1840034016,0.347,,,,,,
8,Los Angeles,Los Angeles,34.1139,-118.4068,United States,US,USA,California,NaN,12815475,1840020491,0.175,,,,,,
24,Chicago,Chicago,41.8373,-87.6862,United States,US,USA,Illinois,NaN,8675982,1840000494,0.254,,,,,,
40,Miami,Miami,25.7839,-80.2102,United States,US,USA,Florida,NaN,6381966,1840015149,0.565,,,,,,
42,Dallas,Dallas,32.7936,-96.7662,United States,US,USA,Texas,NaN,5733259,1840019440,0.317,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15023,Clifton,Clifton,33.0249,-109.2883,United States,US,USA,Arizona,NaN,5010,1840022949,,,,,,,
15024,Sabattus,Sabattus,44.0950,-70.0662,United States,US,USA,Maine,NaN,5009,1840052984,,,,,,,
15025,Sugarcreek,Sugarcreek,41.4380,-79.8183,United States,US,USA,Pennsylvania,NaN,5009,1840003430,,,,,,,
15026,Charlestown,Charlestown,43.2469,-72.3939,United States,US,USA,New Hampshire,NaN,5008,1840054717,,,,,,,


In [52]:
usa_df.dtypes

city           object
city_ascii     object
lat           float64
lng           float64
country        object
iso2           object
iso3           object
state          object
capital        object
population      int64
id              int64
CO             object
NO2            object
OZONE          object
PM10           object
PM25           object
SO2            object
AQI            object
dtype: object

In [ ]:
base_url = "http://api.openweathermap.org/data/2.5/air_pollution?"



In [34]:
base_url = "https://api.ambeedata.com/latest/by-lat-lng?"

headers = {
    'x-api-key': "90ea0eb1cf8e265e3d079017cc7119654b1f623c9c327b9c18ad7d0834b16356",
    'Content-type': "application/json"
    }

for index, row in city_df.iterrows():
    lat = row["lat"]
    lng = row["lng"]
    
    url = f"{base_url}lat={lat}&lng={lng}"
    
    response = requests.get(url, headers=headers)
    print(response)
    
    try:
        city_df.loc[index, "CO"] = response["stations"][0]["CO"]
        
    except Exception as e:
        print(f"{e.args} not found for {city_df.loc[index, 'city']}")
    


<Response [422]>
("'Response' object is not subscriptable",) not found for Tokyo
<Response [422]>
("'Response' object is not subscriptable",) not found for New York
<Response [422]>
("'Response' object is not subscriptable",) not found for Mexico City
<Response [422]>
("'Response' object is not subscriptable",) not found for Mumbai
<Response [422]>
("'Response' object is not subscriptable",) not found for São Paulo
<Response [422]>
("'Response' object is not subscriptable",) not found for Delhi
<Response [422]>
("'Response' object is not subscriptable",) not found for Shanghai
<Response [422]>
("'Response' object is not subscriptable",) not found for Kolkata
<Response [422]>
("'Response' object is not subscriptable",) not found for Los Angeles
<Response [422]>
("'Response' object is not subscriptable",) not found for Dhaka
<Response [422]>
("'Response' object is not subscriptable",) not found for Buenos Aires
<Response [422]>
("'Response' object is not subscriptable",) not found for Ka

KeyboardInterrupt: 

In [8]:
import http.client

conn = http.client.HTTPSConnection("api.ambeedata.com")

headers = {
    'x-api-key': "90ea0eb1cf8e265e3d079017cc7119654b1f623c9c327b9c18ad7d0834b16356",
    'Content-type': "application/json"
    }

conn.request("GET", "/latest/by-lat-lng?lat=12.9889055&lng=77.574044", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"message":"success","stations":[{"CO":0.463,"NO2":13.974,"OZONE":7.898,"PM10":64.468,"PM25":9.119,"SO2":5.365,"city":"Bengaluru","countryCode":"IN","division":"Bengaluru","lat":12.98912,"lng":77.574985,"placeName":"Nagappa Road","postalCode":"560020","state":"Karnataka","updatedAt":"2023-07-19T03:00:00.000Z","AQI":56,"aqiInfo":{"pollutant":"PM10","concentration":64.468,"category":"Moderate"}}]}


In [35]:
import http.client

conn = http.client.HTTPSConnection("api.ambeedata.com")

headers = {
    'x-api-key': "90ea0eb1cf8e265e3d079017cc7119654b1f623c9c327b9c18ad7d0834b16356",
    'Content-type': "application/json"
    }


for index, row in city_df.iterrows():
    lat = row["lat"]
    lng = row["lng"]
    
    conn.request("GET", f"/latest/by-lat-lng?lat={lat}&lng={lng}", headers=headers)

    res = conn.getresponse()
    data = res.read()

    print(data.decode("utf-8"))
    
    try:
        city_df.loc[index, "CO"] = response["stations"][0]["CO"]
        
    except Exception as e:
        print(f"{e.args} not found for {city_df.loc[index, 'city']}")

You have reached the daily limit for the current plan
("'Response' object is not subscriptable",) not found for Tokyo
You have reached the daily limit for the current plan
("'Response' object is not subscriptable",) not found for New York
You have reached the daily limit for the current plan
("'Response' object is not subscriptable",) not found for Mexico City
You have reached the daily limit for the current plan
("'Response' object is not subscriptable",) not found for Mumbai
You have reached the daily limit for the current plan
("'Response' object is not subscriptable",) not found for São Paulo
You have reached the daily limit for the current plan
("'Response' object is not subscriptable",) not found for Delhi
You have reached the daily limit for the current plan
("'Response' object is not subscriptable",) not found for Shanghai
You have reached the daily limit for the current plan
("'Response' object is not subscriptable",) not found for Kolkata
You have reached the daily limit for 

KeyboardInterrupt: 